# PBMC Cell Type Markers



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scanpy.api as sc
import scipy as sp
import itertools
import numpy as np
import scipy.stats as stats
from scipy.integrate import dblquad
import seaborn as sns
from statsmodels.stats.multitest import fdrcorrection
import imp
import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('/wynton/group/ye/mincheol/Github/scrna-parameter-estimation/simplesc')
import scme

In [4]:
data_path = '/wynton/group/ye/mincheol/parameter_estimation/interferon_data/'

### Read the dataset

In [5]:
adata = sc.read(data_path + 'interferon.raw.h5ad')
adata = adata[:, adata.var.index.map(lambda x: x[:2] != 'HB')]
adata = adata[adata.obs['stim'] == 'ctrl', :].copy()

### Load the estimator

In [7]:
imp.reload(scme)
estimator = scme.SingleCellEstimator(
    adata=adata, 
    group_label='cell',
    n_umis_column='n_counts',
    num_permute=10000,
    beta=0.1)

In [ ]:
estimator.load()